In [1]:
import os

In [2]:
%pwd

'd:\\AI ML Project\\oral-lesion-detection-ai\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\AI ML Project\\oral-lesion-detection-ai'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path : Path
    training_data : Path
    params_is_augmentation : bool
    params_image_size : list
    params_epochs : int
    params_batch_size : int


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Oral Cancer Image Dataset")

        create_directories([training.root_dir])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE,
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config:TrainingConfig):
        self.config = config


    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./225,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator



        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)




    def train(self):
        self.steps_per_epoch= max(1, self.train_generator.samples // self.config.params_batch_size)
        self.validation_steps= max(1, self.valid_generator.samples // self.config.params_batch_size)

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_data= self.valid_generator,
            validation_steps= self.validation_steps
        )


        self.save_model(
            path = self.config.trained_model_path,
            model= self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e   

[2025-07-29 18:45:28,717: INFO: common: yaml file : config\config.yaml loaded succesfully]
[2025-07-29 18:45:28,720: INFO: common: yaml file : params.yaml loaded succesfully]
[2025-07-29 18:45:28,721: INFO: common: created directory at: artifacts]
[2025-07-29 18:45:28,723: INFO: common: created directory at: artifacts/training]
Found 188 images belonging to 2 classes.
Found 752 images belonging to 2 classes.


c:\Anaconda\envs\oral\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - Accuracy: 0.5410 - loss: 12.7689

c:\Anaconda\envs\oral\Lib\site-packages\PIL\Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


47/47 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - Accuracy: 0.5410 - loss: 12.7958 - val_Accuracy: 0.5568 - val_loss: 14.2728
